In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
with open("gameData.json") as f:
    data = f.read()

In [3]:
data = json.loads(data)

In [4]:
data["code"], data["createdBy"], data["createdTimestamp"], data["currentTurn"], data["id"], data["playerCount"], data["status"]

('2CEM4E',
 'b65626d0-109a-45eb-ae68-9a08a34db453',
 '2023-06-03T11:54:33.201Z',
 'efd90d8c-9fb5-46ff-9ee6-f5af60752b36',
 'zxkxx5zfowygsj0xoe5kp9dj',
 6,
 'IN_PROGRESS')

In [5]:
encodings = {
    "team" : {
        "Team Bot" : 0,
        "Team Human" : 1
    },
    "players" : {
        0 : {
            "3aebe0fd-5569-4ff8-b1ec-59a0e8d83a6f" : 0,
            "b65626d0-109a-45eb-ae68-9a08a34db453" : 1,
            "6aaacb16-3eee-4fb3-9ef7-0a47302d2cc4" : 2
        },
        1 : {
            "34192b70-263a-4d1e-86d1-35c1a5ec1ac6" : 0,
            "ec4faa9b-9e42-4439-a680-b56620ea28c6" : 1,
            "efd90d8c-9fb5-46ff-9ee6-f5af60752b36" : 2
        },
    },
    "players_map" : {
        "3aebe0fd-5569-4ff8-b1ec-59a0e8d83a6f" : 0,
        "b65626d0-109a-45eb-ae68-9a08a34db453" : 1,
        "6aaacb16-3eee-4fb3-9ef7-0a47302d2cc4" : 2,
        "34192b70-263a-4d1e-86d1-35c1a5ec1ac6" : 3,
        "ec4faa9b-9e42-4439-a680-b56620ea28c6" : 4,
        "efd90d8c-9fb5-46ff-9ee6-f5af60752b36" : 5
    },
    "player_name" : {
        "34192b70-263a-4d1e-86d1-35c1a5ec1ac6" : "Yash Gupta",
        "3aebe0fd-5569-4ff8-b1ec-59a0e8d83a6f" : "Pratik Manghwani",
        "6aaacb16-3eee-4fb3-9ef7-0a47302d2cc4" : "Swarnava Das",
        "b65626d0-109a-45eb-ae68-9a08a34db453" : "Vijayabharathi Murugan",
        "ec4faa9b-9e42-4439-a680-b56620ea28c6" : "Yash Khivasara",
        "efd90d8c-9fb5-46ff-9ee6-f5af60752b36" : "Preetam Sekhar"
    },
    "game_id" : "zxkxx5zfowygsj0xoe5kp9dj"
}


In [6]:
set_encoding = {
    "suit" : {
        0 : "lower_hearts",
        1 : "lower_diamonds",
        2 : "lower_spades",
        3 : "lower_clubs",
        4 : "upper_hearts",
        5 : "upper_diamonds",
        6 : "upper_spades",
        7 : "upper_clubs",
    },
    "set_id_type" : {
        0 : "lower",
        1 : "lower",
        2 : "lower",
        3 : "lower",
        4 : "upper",
        5 : "upper",
        6 : "upper",
        7 : "upper",
    },
    "card_id" : {
        "lower" : {
            0 : "ace",
            1 : "two",
            2 : "three",
            3 : "four",
            4 : "five",
            5 : "six",
        },
        "upper" : {
            0 : "eight",
            1 : "nine",
            2 : "ten",
            3 : "jack",
            4 : "queen",
            5 : "king",
        }
    }
}

In [7]:
game_array = np.full((8,6), -1)

In [8]:
def infer_ids(card_dict):
    rank = card_dict["rank"].lower()
    suit = card_dict["suit"].lower()
    set_id_type = [i for i,j in set_encoding["card_id"].items() if rank in j.values()][0]
    set_id = [i for i,j in set_encoding["suit"].items() if set_id_type+"_"+suit == j][0]
    card_id = [i for i,j in set_encoding["card_id"][set_id_type].items() if rank == j][0]

    return set_id, card_id

In [9]:
for move in data["moves"]:
    if (move["actionData"]["action"] == "ASK") and move["resultData"]["success"]:

        card_dict = move["actionData"]["askData"]["card"]
        set_id, card_id = infer_ids(card_dict)

        if set_id == 3:
            pass
        else:
            ask_player_id = encodings["players_map"][move["actionData"]["askData"]["by"]]
            ans_player_id = encodings["players_map"][move["actionData"]["askData"]["from"]]

            # assert game_array[set_id, card_id] == ask_player_id
            game_array[set_id, card_id] = ans_player_id
        
    elif (move["actionData"]["action"] == "CALL_SET") and move["resultData"]["success"]:

        if move["actionData"]["callData"]["set"].lower() == "small clubs":
            pass
        else:
            for player_id, card_list in move["actionData"]["callData"]["data"].items():
                for card_dict in card_list:
                    set_id, card_id = infer_ids(card_dict)

                    game_array[set_id, card_id] = encodings["players_map"][player_id]
    else:
        pass

In [10]:
k = []
for move in data["moves"]:
    if (move["actionData"]["action"] == "ASK"):

        card_dict = move["actionData"]["askData"]["card"]
        set_id, card_id = infer_ids(card_dict)

        if set_id == 3:
            k.append(move)
       
    elif (move["actionData"]["action"] == "CALL_SET"):

        if move["actionData"]["callData"]["set"].lower() == "small clubs":
            k.append(move)
    else:
        pass

In [11]:
game_array[3, :] = [1, 0, 2, 0, 1, 2]

In [16]:
action_dict_list = {}
count = -1
for move in data["moves"][::-1]:
    if (move["actionData"]["action"] == "ASK"):

        card_dict = move["actionData"]["askData"]["card"]
        set_id, card_id = infer_ids(card_dict)

        ask_player_id = encodings["players_map"][move["actionData"]["askData"]["by"]]
        ans_player_id = encodings["players_map"][move["actionData"]["askData"]["from"]]

        # assert game_array[set_id, card_id] == ask_player_id
        action_dict = {
            "action" : "ask_card",
            "by_team" : 0 if ask_player_id>=3 else 1,
            "by" : ask_player_id-3 if ask_player_id>=3 else ask_player_id,
            "to_team" : 0 if ans_player_id>=3 else 1,
            "to" : ans_player_id-3 if ans_player_id>=3 else ans_player_id,
            "set_id" : set_id,
            "card_id" : card_id,
            "result" : 1 if move["resultData"]["success"] else 0
        }

        count += 1
        action_dict_list = action_dict_list | {count : action_dict}
        
    elif (move["actionData"]["action"] == "CALL_SET"):

        pass
        # if move["actionData"]["callData"]["set"].lower() == "small clubs":
        #     pass
        # else:
        #     for player_id, card_list in move["actionData"]["callData"]["data"].items():
        #         for card_dict in card_list:
        #             set_id, card_id = infer_ids(card_dict)

        #             game_array[set_id, card_id] = encodings["players_map"][player_id]
    else:
        pass

In [31]:
action_data_json = {
    "game_id" : data["id"],
    "game_code" : data["code"],
    "player_count" : data["playerCount"],
    "actions" : action_dict_list,
    "card_location_array" : game_array.tolist()
}

In [32]:
with open("gameActionData.json", mode="w") as f:
    f.write(json.dumps(action_data_json))